In [13]:
import keras
from keras.models import load_model


import sqmutils.data_utils as du

import os
import time 

import pandas as pd

import json

%load_ext autoreload
%autoreload 2
%matplotlib inline 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model_dir = "models"
model_weights = os.path.join(model_dir, "best_val_f1_model.h5")
test_dataset_path = "/home/elkhand/Downloads/test.csv"
embedding_path = "/home/elkhand/datasets/fasttext/wiki.en.vec"
emb_dim = 300


config = du.get_config(None, None, None,  embedding_dimension=emb_dim)


custom_objects= {"f1": du.f1, "recall" : du.recall, "precision" : du.precision}


#model = mu._build_model(config, model_dir, drop_rate=0.75, isParallel=False)

model = load_model(model_weights, custom_objects = custom_objects)

config
 {'train_dataset_path': None, 'test_size': None, 'val_size': None, 'max_seq_len': 32, 'embedding_dimension': 300, 'batch_size': 3096, 'nb_epochs': 100, 'recurrent_dropout': 0.3, 'dropout': 0.3, 'seed': 7, 'is_debug_on': False} 



# Reading test data

In [4]:
start = time.time()
dfTest = pd.read_csv(test_dataset_path, sep=',', encoding='utf-8')
end = time.time()
print("Total time passed", (end - start))

/home/elkhand/anaconda3/envs/cs231n/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total time passed 5.631618022918701


In [7]:
dfTest[:10]

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
5,5,How are the two wheeler insurance from Bharti ...,I admire I am considering of buying insurance ...
6,6,How can I reduce my belly fat through a diet?,How can I reduce my lower belly fat in one month?
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...
8,8,What are the how best books of all time?,What are some of the military history books of...
9,9,After 12th years old boy and I had sex with a ...,Can a 14 old guy date a 12 year old girl?


# Load embeddings

We will be using Fasttext Wiki word vectors 300D

In [8]:
print("word vectors path", embedding_path)
start = time.time()
w2v = du.load_embedding(embedding_path)
end = time.time()
print("Total time passed: ", (end-start))

word vectors path /home/elkhand/datasets/fasttext/wiki.en.vec
embedding size : 2519371
embedding dimension : (300,)
Total time passed:  326.8346252441406


# Convert Test dataset into word vectors

In [9]:
limit = 100

start = time.time()
df_test_q1_emb,df_test_q2_emb  = du.load_dataset(dfTest[:limit], w2v, config, isTestDataset=True)
end = time.time()
print("Total time passed", (end - start))

Total time passed 0.0715322494506836


# Evaluate test dataset

In [15]:
# evaluate loaded model on test data 
# Define X_test & Y_test data first
optimizer = keras.optimizers.Nadam()
model.compile(loss='binary_crossentropy', optimizer=optimizer, 
                  metrics=['accuracy', du.f1, du.recall, du.precision])
#score = loaded_model.evaluate(X_test, Y_test, verbose=0)
#print ("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
#result = model.evaluate(x=[df_test_q1_emb, df_test_q2_emb], y=df_test_label)

result = model.predict([df_test_q1_emb, df_test_q2_emb], verbose=1) #, batch_size=None, steps=None
print(result)


100/100 [==============================] - 3s 33ms/step
[[2.74974197e-01]
 [1.21576197e-01]
 [2.39591405e-01]
 [3.07962619e-04]
 [7.03492761e-02]
 [1.85518269e-03]
 [9.10502911e-01]
 [9.42221045e-01]
 [3.40958238e-01]
 [5.35632810e-03]
 [6.45618320e-01]
 [2.97361566e-06]
 [2.64778789e-11]
 [3.71955000e-02]
 [3.39044780e-01]
 [1.55428305e-01]
 [1.51343236e-04]
 [7.17056453e-01]
 [1.11396126e-01]
 [1.37341201e-01]
 [2.36008869e-04]
 [7.00133678e-05]
 [2.60508627e-01]
 [4.39848900e-01]
 [2.18836349e-02]
 [7.71729469e-01]
 [1.96059234e-04]
 [2.08868762e-04]
 [5.62036657e-05]
 [5.00048627e-05]
 [9.78877768e-04]
 [3.38044512e-04]
 [9.21797976e-02]
 [6.06054924e-02]
 [3.27766663e-03]
 [4.39509571e-01]
 [9.36554534e-06]
 [1.67222340e-02]
 [9.46590126e-01]
 [1.59650762e-02]
 [8.78638506e-01]
 [7.44523704e-01]
 [4.36807752e-01]
 [6.19019265e-04]
 [2.71081983e-04]
 [2.80758411e-01]
 [8.90025914e-01]
 [5.64279616e-01]
 [8.57277989e-01]
 [5.43361704e-04]
 [5.28701302e-03]
 [2.59911031e-01]
 [1.5590